In [1]:
#!pip install scipy

In [2]:
import pandas as pd
import numpy as np
import re

In [7]:
fn_files = ['cf74','cf75','cf76','cf77', 'cf78', 'cf79']
fn_queries = 'cfquery' 

In [14]:
def extract_informations(filename):
    data = {}  
    id_actual = None  
    information = ""  

    try:
        with open(filename, "r", encoding='ansi') as file:
            for line in file:
                if line.startswith("PN"):
                    # Se há uma informação anterior, armazena no dicionário
                    if id_actual and information:
                        data[id_actual] = information
                        information = ""  

                    id_actual = line.split()[1]  # Obtém o ID a partir da segunda palavra
                elif line.startswith(("TI", "MJ", "MN", "AB", "EX")) and id_actual:
                    information += line[3:].strip()
                    while True:
                        next_line = file.readline()
                        if not next_line or re.match(r'\b[A-Z]{2}\s', next_line):
                            break
                        information += " " + next_line.strip()

            # Adiciona a última informação ao dicionário
            if id_actual and information:
                data[id_actual] = information

    except FileNotFoundError:
        print("File not founded.")

    return data

# Dicionário para armazenar os data
data_complete = {}

# Extrai as informações de cada file
for fn in fn_files:
    path_file = f"../data/{fn}"  
    data = extract_informations(path_file)

    data_complete.update(data)

data

{'79001': "'Rusters'.  The corrosive action of palmar sweat: I. Sodium chloride in sweat.CORROSION.  HYPERHIDROSIS.  METALLURGY.  OCCUPATIONAL-MEDICINE. SODIUM-CHLORIDE.  SWEAT: an.Sweat from 8 'rusters' and 8 control persons was examined for its sodium concentration. Sweating was induced by iontophoresis of pilocarpine on the distal forearm. In no case did the concentrations exceed the normal upper limit for this procedure, and no difference was found between the two groups. In this way the study failed to confirm earlier reports of elevated sodium chloride concentrations in sweat from 'rusters'. Proposals of a relationship between 'rusters' and patients with pancreatic cystic fibrosis were not substantiated. Hyperhidrosis of the palms and volar surfaces of the fingers was present in all 'rusters'. This seemed to be the main cause of the corrosive tendency, and patients referred for palmar hyperhidrosis were found to produce corrosion similar to the 'rusters'. Topical application of a

In [11]:
def read_queries(file_path):

    qn_list = []
    qu_list = []
    nr_list = []
    rd_list = []

    # Lê o file e processa as lines
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('QN'):
                qn_list.append(line.split()[1])
            elif line.startswith('QU'):
                qu_list.append(' '.join(line.split()[1:]))
            elif line.startswith('NR'):
                nr_list.append(line.split()[1])
            elif line.startswith('RD'):
                # Separa os valores de RD e os armazena como uma lista
                rd_list.append([int(x) for x in line.split()[1:]])

    # Cria um dicionário com as listas
    data = {
        'QN': qn_list,
        'QU': qu_list,
        'NR': nr_list,
        'RD': rd_list
    }

    # Cria o DataFrame
    df = pd.DataFrame(data)
    return df

df_queries = read_queries(f'../data/{fn_queries}')

In [12]:
df_queries

,QN,QU,NR,RD
0,00001,What are the effects of calcium on the physica...,00034,"[139, 1222, 151, 2211, 166, 1, 311, 1, 370, 10..."
1,00002,Can one distinguish between the effects of muc...,00007,"[169, 1000, 434, 1001, 454, 100, 498, 1000, 49..."
2,00003,How are salivary glycoproteins from CF patient...,00043,"[23, 1000, 40, 10, 139, 2122, 190, 1, 221, 1, ..."
3,00004,What is the lipid composition of CF respirator...,00009,"[503, 1, 538, 100, 539, 100, 540, 100, 553, 1,..."
4,00005,Is CF mucus abnormal?,00131,"[23, 2220, 47, 2221, 50, 1, 60, 1, 114, 11, 13..."
...,...,...,...,...
95,00096,Is there an increased incidence of dental prob...,00012,"[9, 1, 40, 1001, 43, 100, 75, 2212, 454, 2222,..."
96,00097,Is oxygen transport by red blood cells abnorma...,00011,"[52, 100, 68, 1000, 135, 1, 140, 101, 190, 1, ..."
97,00098,What are the effects of CF on the development ...,00015,"[145, 2221, 180, 2120, 270, 1, 272, 2210, 362,..."
98,00099,Are there abnormalities of taste in CF patients?,00005,"[268, 2222, 324, 2222, 449, 2122, 992, 1220, 1..."
